# Project: **Newton-Raphson** in **GridCal**

Created by Tiancheng Shao. This notebook is a blank slate for you to write in.  Feel free to include figures (don't forget to add/commit them to your repository) and examples.  You can change the kernel (from `Python 3`; see upper right) if the open source project you're writing about does not use Python.  You can write from the prompts below or delete all the cells and start fresh.  Note that Git will always contain your history.


## About the method

We have learned Newton-Raphson method in rootfinding topic of our course. It's a method for finding the roots of differentiable functions

![](https://render.githubusercontent.com/render/math?math=x%20%3D%20x_0%20-%20f%28x_0%29%20%2F%20f%27%28x_0%29%20.&mode=display)

## About the software

![](https://raw.githubusercontent.com/SanPen/GridCal/master/pics/GridCal_banner.png)
https://github.com/SanPen/GridCal 

Gridcal is a research oriented power systems software. The developer of GridCal is Santiago Peñate Vera, he's a power systems researcher with plentiful project experience.

This software aims to be a complete platform for power systems research and simulation. People who are engaged in power system related occupations can get help from it.

It is written by Python, and it can be used with a GUI or as a library.

![](https://raw.githubusercontent.com/SanPen/GridCal/master/pics/GridCal.png)

## Newton-Raphson appears in Gridcal

Newton-Raphson method was implemented in function **IwamotoNR**. It's a method to solve power flow using a full Newton's method with the Iwamoto optimal step factor.

The function is listed at the end of page

![](https://raw.githubusercontent.com/fxt0706/GridCalChineseWiki/master/docs/guide/pics/ShowPowerFlowSolver.png)

### Canonical Newton-Raphson

Gridcal has made same modifications to the original Newton's method to turn it to a more robust, industry-standard algorithm.

The expression to update the voltage solution in the Newton-Raphson algorithm:

![](https://gridcal.readthedocs.io/en/latest/_images/math/57c068ddf5402ef94b25afee43e5d9b0fb54cc42.png)

### Newton-Raphson-Iwamoto

The Iwamoto and Tamura’s modification to the Newton-Raphson algorithm consist in finding an optimal acceleration parameter µ that determines the length of the iteration step:

![](https://gridcal.readthedocs.io/en/latest/_images/math/9aa08221f9b30bf79e773175ded70d14ff7c773b.png)



In [3]:
def IwamotoNR(Ybus, Sbus, V0, Ibus, pv, pq, tol, max_it=15, robust=False):
    """
    Solves the power flow using a full Newton's method with the Iwamoto optimal step factor.
    Args:
        Ybus: Admittance matrix
        Sbus: Array of nodal power injections
        V0: Array of nodal voltages (initial solution)
        Ibus: Array of nodal current injections
        pv: Array with the indices of the PV buses
        pq: Array with the indices of the PQ buses
        tol: Tolerance
        max_it: Maximum number of iterations
        robust: Boolean variable for the use of the Iwamoto optimal step factor.
    Returns:
        Voltage solution, converged?, error, calculated power injections
    @author: Ray Zimmerman (PSERC Cornell)
    @Author: Santiago Penate Vera
    """
    start = time.time()

    # initialize
    converged = 0
    iter_ = 0
    V = V0
    Va = np.angle(V)
    Vm = np.abs(V)
    dVa = np.zeros_like(Va)
    dVm = np.zeros_like(Vm)

    # set up indexing for updating V
    pvpq = np.r_[pv, pq]
    npv = len(pv)
    npq = len(pq)

    # j1:j2 - V angle of pv buses
    j1 = 0
    j2 = npv + npq
    # j2:j3 - V mag of pq buses
    j3 = j2 + npq

    if (npq + npv) > 0:

        # generate lookup pvpq -> index pvpq (used in createJ)
        pvpq_lookup = np.zeros(np.max(Ybus.indices) + 1, dtype=int)
        pvpq_lookup[pvpq] = np.arange(len(pvpq))
        createJ = get_fastest_jacobian_function(pvpq, pq)

        # evaluate F(x0)
        Scalc = V * np.conj(Ybus * V - Ibus)
        mis = Scalc - Sbus  # compute the mismatch
        f = np.r_[mis[pvpq].real, mis[pq].imag]

        # check tolerance
        norm_f = np.linalg.norm(f, np.Inf)

        if norm_f < tol:
            converged = 1

        # do Newton iterations
        while not converged and iter_ < max_it:
            # update iteration counter
            iter_ += 1

            # evaluate Jacobian
            # J = Jacobian(Ybus, V, Ibus, pq, pvpq)
            J = _create_J_with_numba(Ybus, V, pvpq, pq, createJ, pvpq_lookup, npv, npq)

            # compute update step
            try:
                dx = linear_solver(J, f)
            except:
                print(J)
                converged = False
                iter_ = max_it + 1  # exit condition

            # reassign the solution vector
            dVa[pvpq] = dx[j1:j2]
            dVm[pq] = dx[j2:j3]
            dV = dVm * np.exp(1j * dVa)  # voltage mismatch

            # update voltage
            if robust:
                # if dV contains zeros will crash the second Jacobian derivative
                if not (dV == 0.0).any():
                    # calculate the optimal multiplier for enhanced convergence
                    mu_ = mu(Ybus, Ibus, J, f, dV, dx, pvpq, pq, npv, npq)
                else:
                    mu_ = 1.0
            else:
                mu_ = 1.0

            Vm -= mu_ * dVm
            Va -= mu_ * dVa

            V = Vm * np.exp(1j * Va)

            Vm = np.abs(V)  # update Vm and Va again in case
            Va = np.angle(V)  # we wrapped around with a negative Vm

            # evaluate F(x)
            Scalc = V * np.conj(Ybus * V - Ibus)
            mis = Scalc - Sbus  # complex power mismatch
            f = np.r_[mis[pvpq].real, mis[pq].imag]  # concatenate again

            # check for convergence
            norm_f = np.linalg.norm(f, np.Inf)

            if norm_f < tol:
                converged = 1
    else:
        norm_f = 0
        converged = True
        Scalc = Sbus

    end = time.time()
    elapsed = end - start

    return V, converged, norm_f, Scalc, iter_, elapsed

## Source

https://gridcal.readthedocs.io/en/latest/theory/models.html